# **Fantasy Sports League Optimization**

## **Project Description**
This project focuses on optimizing team assignments in a fantasy sports league while ensuring a balanced distribution of talent and adherence to salary constraints. Given a dataset of players with attributes such as skill rating, cost, and position, we aim to assign them to five teams while following strict constraints and achieving a balanced league.

## **Constraints**
- Each team must consist of:
  - 1 Goalkeeper (GK)  
  - 2 Defenders (DEF)  
  - 2 Midfielders (MID)  
  - 2 Forwards (FWD)  
- Each player is assigned to exactly one team.  
- No team may exceed a total budget of **750 million €**.  
- The standard deviation of the average skill rating of all teams should be minimized to ensure fairness.  

## **Objective**
To generate a valid league configuration that follows all constraints and ensures that teams have a similar overall skill level, measured by the standard deviation of their average skill ratings.

## **Group Members**

### **Group X**

| Name                  | Student Number  |
|-----------------------|----------------|
| Philippe Dutranoit   | 20240518        |
| Name 2               | Student Number 2 |
| Name 3               | Student Number 3 |
| Name 4               | Student Number 4 |

# Imports 

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
df = pd.read_csv('../Data/players(in).csv')

In [ ]:
df.drop(columns=['Unnamed: 0'], inplace=True) # drop a column that as no information 
df.hea


,Name,Position,Skill,Salary (€M)
0,Alex Carter,GK,85,90
1,Jordan Smith,GK,88,100
2,Ryan Mitchell,GK,83,85
3,Chris Thompson,GK,80,80
4,Blake Henderson,GK,87,95


# Defining The constraints

In [12]:
position = {"GK": 1, "DEF": 2, "MID": 2, "FWD": 2}
budget = 750 
total_players = 5 # as 35 rows devided by seven position = 5 teams
team_size = sum(position.values())

In [ ]:
# create a dictionary to group players by position
position_groups = {pos: df[df["Position"] == pos].values.tolist() for pos in position}

position_groups

{'GK': [['Alex Carter', 'GK', 85, 90],
  ['Jordan Smith', 'GK', 88, 100],
  ['Ryan Mitchell', 'GK', 83, 85],
  ['Chris Thompson', 'GK', 80, 80],
  ['Blake Henderson', 'GK', 87, 95]],
 'DEF': [['Daniel Foster', 'DEF', 90, 110],
  ['Lucas Bennett', 'DEF', 85, 90],
  ['Owen Parker', 'DEF', 88, 100],
  ['Ethan Howard', 'DEF', 80, 70],
  ['Mason Reed', 'DEF', 82, 75],
  ['Logan Brooks', 'DEF', 86, 95],
  ['Caleb Fisher', 'DEF', 84, 85],
  ['Maxwell Flores', 'DEF', 81, 72],
  ['Jaxon Griffin', 'DEF', 79, 65],
  ['Brayden Hughes', 'DEF', 87, 100]],
 'MID': [['Nathan Wright', 'MID', 92, 120],
  ['Connor Hayes', 'MID', 89, 105],
  ['Dylan Morgan', 'MID', 91, 115],
  ['Hunter Cooper', 'MID', 83, 85],
  ['Austin Torres', 'MID', 82, 80],
  ['Gavin Richardson', 'MID', 87, 95],
  ['Spencer Ward', 'MID', 84, 85],
  ['Bentley Rivera', 'MID', 88, 100],
  ['Dominic Bell', 'MID', 86, 95],
  ['Ashton Phillips', 'MID', 90, 110]],
 'FWD': [['Sebastian Perry', 'FWD', 95, 150],
  ['Xavier Bryant', 'FWD', 90, 